In [1]:
# imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')

import re

import string

from sklearn.pipeline import make_pipeline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.svm import LinearSVC

from sklearn.feature_selection import SelectKBest

from sklearn.decomposition import TruncatedSVD

from sklearn import metrics

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import altair as alt

import joblib

In [2]:
# we're going to train a quick model on a set of bitcoin tweets, labeled with their sentiment.
tweets = pd.read_csv('../data/btc_tweets_daily_example.csv')
tweets.head()

,column_a,date,tweet,screen_name,source,link,sentiment,sent_score,new_sentiment_score,new_sentiment_state
0,0,Fri Mar 23 00:40:32 +0000 2018,"RT @ALXTOKEN: Paul Krugman, Nobel Luddite. I h...",myresumerocket,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['neutral'],0,0.000000,0
1,1,Fri Mar 23 00:40:34 +0000 2018,@lopp @_Kevin_Pham @psycho_sage @naval But @Pr...,BitMocro,[u'Bitcoin'],"<a href=""http://twitter.com/download/android"" ...",['neutral'],0,0.000000,0
2,2,Fri Mar 23 00:40:35 +0000 2018,RT @tippereconomy: Another use case for #block...,hojachotopur,"[u'blockchain', u'Tipper', u'TipperEconomy']","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['positive'],1,0.136364,1
3,3,Fri Mar 23 00:40:36 +0000 2018,free coins https://t.co/DiuoePJdap,denies_distro,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['positive'],1,0.400000,1
4,4,Fri Mar 23 00:40:36 +0000 2018,RT @payvxofficial: WE are happy to announce th...,aditzgraha,[],"<a href=""http://twitter.com/download/android"" ...",['positive'],1,0.468182,1


In [3]:
# Dropping unused columns, checking shape

tweets.drop(columns=['column_a','source','screen_name','date','link','sentiment',
             'new_sentiment_score','new_sentiment_state'], inplace=True)

print(tweets.shape)

(50859, 2)


In [4]:
def clean_text(text): 
    """Clean our tweets using regex. Remove weird characters and links."""
    cleanedup = text.lower()
    return re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", cleanedup)

In [5]:
# clean the tweets, divide into feature and target, divide into train and test

tweets['tweet'] = tweets['tweet'].apply(clean_text)

train, test = train_test_split(tweets, test_size=0.2, stratify=tweets['sent_score'],
                              random_state = 42)

X_train = train['tweet']
X_test = test['tweet']

y_train = train['sent_score']
y_test = test['sent_score']

In [6]:
# make a pipeline for processing tweets. 
# min_df and max_df are statistical trimming parameters, they get rid of grams below and above certain frequencies.
process_pipe = make_pipeline(
            CountVectorizer(ngram_range=(1,2), max_df=0.95, min_df=10),
            SelectKBest(k=5000)
)

In [7]:
# process our data
X_train_processed = process_pipe.fit_transform(X_train,y_train)
X_test_processed = process_pipe.transform(X_test)

In [8]:
# Get the list of features that were kept
all_names = pd.Series(process_pipe.steps[0][1].get_feature_names())
selected_mask = pd.Series(process_pipe.steps[1][1].get_support())
selected_names = all_names[selected_mask]

In [9]:
# re-DataFrame the data and add back in columns
X_train_kbest_df = pd.DataFrame(X_train_processed.toarray(), columns=selected_names)
X_test_kbest_df = pd.DataFrame(X_test_processed.toarray(), columns=selected_names)

In [10]:
X_train_kbest_df.head()

nomists  went live  were  \
0        0         0          0                     0          0     0   
1        0         0          0                     0          0     0   
2        0         0          0                     0          0     0   
3        0         0          0                     0          0     0   
4        0         0          0                     0          0     0   

   were bringing  were celebrating  were doing  were finally  were involving  \
0              0                 0           0             0               0   
1              0                 0           0             0               0   
2              0                 0           0             0               0   
3              0                 0           0             0               0   
4              0                 0           0             0               0   

   were launching  were revealing  were told  weve  weve achieved  \
0               0               0          0     0              0   
1               0               0          0     0              0   
2               0               0          0     0              0   
3               0               0          0     0              0   
4               0               0          0     0              0   

   weve managed  whale  whale has  what  what are  what do  what does  \
0             0      0          0     0         0        0          0   
1             0      0          0     0         0        0          0   
2             0      0          0     0         0        0          0   
3             0      0          0     0         0        0          0   
4             0      0          0     0         0        0          0   

   what happens  what im  what is  what sport  what they  what were  what you  \
0             0        0        0           0          0          0         0   
1             0        0        0           0          0          0         0   
2             0        0        0           0          0          0         0   
3             0        0        0           0          0          0         0   
4             0        0        0           0          0          0         0   

   whats  whats happening  when in  when the  when transacting  when we  \
0      0                0        0         0                 0        0   
1      0                0        0         0                 0        0   
2      0                0        0         0                 0        0   
3      0                0        0         0                 0        0   
4      0                0        0         0                 0        0   

   when you  where  where could  where everybody  where the  which are  \
0         0      0            0                0          0          0   
1         0      0            0                0          0          0   
2         0      0            0                0          0          0   
3         0      0            0                0          0          0   
4         0      0            0                0          0          0   

   which suchapp  while  while bettingwell  while one  while you  white  \
0              0      0                  0          0          0      0   
1              0      0                  0          0          0      0   
2              0      0                  0          0          0      0   
3              0      0                  0          0          0      0   
4              0      0                  0          0          0      0   

   white airdrop  whitepaper  whitepaper turns  who  who are  who complete  \
0              0           0                 0    0        0             0   
1              0           0                 0    0        0             0   
2              0           0                 0    0        0             0   
3              0           0                 0    0        0             0   
4              0           0              

In [11]:
# we'll use an SVM for classifying this data.
clf = LinearSVC(C = 1.2, random_state = 42)

In [12]:
# Fit the model
clf.fit(X_train_kbest_df,y_train)

LinearSVC(C=1.2, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
          verbose=0)

In [13]:
# Score on the holdout
clf.score(X_test_kbest_df, y_test)

0.9633307117577664

In [14]:
def headsandtails(df, x=10): 
    """get the first and last values of a dataframe."""
    return df.head(x).append(df.tail(x))

In [15]:
# Make dataframes of the feature coefficiens for each class
negative_coefficients = pd.DataFrame({
    'features': selected_names,
    'coefs': (clf.coef_[0]),
})

neutral_coefficients = pd.DataFrame({
    'features': selected_names,
    'coefs': (clf.coef_[1]),
})

positive_coefficients = pd.DataFrame({
    'features': selected_names,
    'coefs': (clf.coef_[2]),
})

In [16]:
# Chart of negative importances
negative_importance = headsandtails(negative_coefficients.sort_values('coefs'))

negative_importance_chart = alt.Chart(
    negative_importance,
    title='20 Most Important Words for Negative Class'
).mark_bar().encode(
    x=alt.X('coefs', title = 'Importance', sort = None),
    y=alt.Y(
        'features',
        title = 'Word',
        sort=alt.EncodingSortField(
            field="coefs",  # The field to use for the sort
            order="descending"  # The order to sort in
        )
    ),
    color=alt.Color('coefs', title=None, scale=alt.Scale(scheme="blueorange")),
    tooltip=[
        alt.Tooltip('coefs', title='Importance'),
    ]
).properties(width=550)

negative_importance_chart

alt.Chart(...)

In [17]:
# Neutral importances
neutral_importance = headsandtails(neutral_coefficients.sort_values('coefs'))

neutral_importance_chart = alt.Chart(
    neutral_importance,
    title='20 Most Important Words for Neutral Class'
).mark_bar().encode(
    x=alt.X('coefs', title = 'Importance', sort = None),
    y=alt.Y(
        'features',
        title = 'Word',
        sort=alt.EncodingSortField(
            field="coefs",  # The field to use for the sort
            order="descending"  # The order to sort in
        )
    ),
    color=alt.Color('coefs', title=None, scale=alt.Scale(scheme="blueorange")),
    tooltip=[
        alt.Tooltip('coefs', title='Importance'),
    ]
).properties(width=550)

neutral_importance_chart


alt.Chart(...)

In [18]:
# Positive importances
positive_importance = headsandtails(positive_coefficients.sort_values('coefs'))

positive_importance_chart = alt.Chart(
    positive_importance,
    title='20 Most Important Words for Positive Class'
).mark_bar().encode(
    x=alt.X('coefs', title = 'Importance', sort = None),
    y=alt.Y(
        'features',
        title = 'Word',
        sort=alt.EncodingSortField(
            field="coefs",  # The field to use for the sort
            order="descending"  # The order to sort in
        )
    ),
    color=alt.Color('coefs', title=None, scale=alt.Scale(scheme="blueorange")),
    tooltip=[
        alt.Tooltip('coefs', title='Importance'),
    ]
).properties(width=550)

positive_importance_chart

alt.Chart(...)

In [19]:
importance_dict = dict(zip(selected_names, clf.coef_[2]))

In [20]:
importance_dict['sick']

-2.423058083754991

In [21]:
def importance_lookup(a_word):
    """returns the importance of a word if it exists, otherwise returns zero."""

    fixed_word = a_word.lower().strip(' ').strip(',').strip('.')
    try: value = importance_dict[a_word]
    except: value = 0

    return round(value, 2)

       

In [22]:
importance_lookup('sick')

-2.42

In [23]:
negative_tweet = 'bitcoin is a ponzi scheme and SEC intervention is on its way.'

In [24]:
text_series = pd.DataFrame(negative_tweet.split(' '), columns=['word'])
text_series['importances'] = text_series['word'].apply(importance_lookup)

In [25]:
text_series

,word,importances
0,bitcoin,-0.07
1,is,0.05
2,a,0.00
3,ponzi,0.00
4,scheme,0.00
5,and,0.06
6,SEC,0.00
7,intervention,0.00
8,is,0.05
9,on,0.02


In [26]:
text_chart = alt.Chart(text_series.reset_index()).mark_bar().encode(
    x=alt.X('importances', title = 'importance', sort = None),
    y=alt.Y('word', title = 'word', sort = None),
    color=alt.Color('importances', scale=alt.Scale(scheme='redblue'))
)

In [27]:
text_chart

alt.Chart(...)

In [29]:
# pickle our pipeline and classifier for later use.
joblib.dump(process_pipe,'../pickles/process_pipe')
joblib.dump(clf,'../pickles/clf')

['../pickles/clf']

In [30]:
joblib.dump(selected_names,'../pickles/feature_list')

['../pickles/feature_list']